In [1]:
import pandas as pd
import numpy as np
from fastai.tabular.all import *

In [26]:
df = pd.read_pickle("df_eda_done.pkl")
sample = pd.read_pickle("sample_eda_done.pkl")

In [29]:
# Variables to use to predict income and wealth1

cont_names = [
    "AGE",
    "IC1",
    "IC2",
    "IC3",
    "IC4",
    "IC5",
    "IC6",
    "IC7",
    "IC8",
    "IC9",
    "IC10",
    "IC11",
    "IC12",
    "IC13",
    "IC14",
    "RAMNTALL",
]

cat_names = [
    "TCODE",
    "STATE",
    "ZIP",
    "MAJOR",
    "MDMAUD",
    "DOMAIN",
    "HOMEOWNR",
    "NUMCHLD",
    "ageGroup",
    "GENDER",
    "WEALTH1",
    "WEALTH2",
    "MSA",
    "ADI",
    "DMA",
    
]

In [30]:
# cont_names, cat_names = cont_cat_split(df, max_card=40)
df_pred = df[cat_names + cont_names + ["cat_INCOME"]].copy()
# df_pred = df[cat_names + cont_names].copy()
train_df = df_pred[~df_pred["cat_INCOME"].isnull()].copy()
# train_df["INCOME"] = train_df["cat_INCOME"]
test_df = df_pred[df_pred["cat_INCOME"].isnull()].drop(["cat_INCOME"], axis=1).copy()

In [32]:
splits = RandomSplitter(valid_pct=0.3)(range_of(train_df))

to = TabularPandas(train_df, procs=[Categorify, FillMissing, Normalize],
                  cat_names = cat_names,
                  cont_names = cont_names,
                  y_names = "cat_INCOME", y_block=CategoryBlock,
                  splits=splits)

dls = to.dataloaders(bs=64)

learn = tabular_learner(dls, metrics=accuracy)

In [33]:
learn.fit_one_cycle(1, 0.03)

epoch,train_loss,valid_loss,accuracy,time
0,0.789886,0.774734,0.639565,00:13


In [34]:
dl = learn.dls.test_dl(test_df)
preds = learn.get_preds(dl=dl, with_decoded=True, with_input=True)

In [44]:
to_np(preds[-1]).mean()

0.5151742929625106

In [36]:
preds[2].size()

AttributeError: 'NoneType' object has no attribute 'size'